In [9]:
import xarray as xr
from windspharm.standard import VectorWind
from windspharm.tools import prep_data, recover_data, order_latdim
from windspharm.examples import example_data_path
import numpy as np

In [12]:
datav925 = xr.open_dataset("v_wmc.nc")
datau925 = xr.open_mfdataset("u_wmc.nc")

In [13]:
lat=datav925.variables['lat'].values
lon=datav925.variables['lon'].values
time=datav925.variables['time'].values
u=datau925['u']
v=datav925['v']

In [14]:
uwnd, uwnd_info = prep_data(u.values, 'tyx')
vwnd, vwnd_info = prep_data(v.values, 'tyx')
lats, uwnd, vwnd = order_latdim(lat, uwnd, vwnd)

In [26]:
w = VectorWind(uwnd, vwnd, gridtype='regular')

In [19]:
sf = w.streamfunction()
sf = recover_data(sf, uwnd_info)

In [22]:
vo = w.vorticity()
vo = recover_data(vo, uwnd_info)

In [20]:
ds = xr.Dataset(
     {"sf10-7": (("time", "lat", "lon"), np.flip(sf,axis=1)/10000000)},
    coords={
    "time": time,
    "lat": lat,
    "lon": lon,
    },
    )

In [24]:
ds.to_netcdf('streamfunction_wmc.nc')